# Exploring higher-level functions in  `PyPestUtils.helpers` with a structured Freyberg model

In [ ]:
import os
import sys
import shutil
import subprocess as sp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Setup the working dir using an existing set of model files

In [ ]:
org_d = "freyberg_monthly"
w_d = "freyberg_highlevel_helpers"
if os.path.exists(w_d):
    shutil.rmtree(w_d)
shutil.copytree(org_d,w_d)

Just some basic viz:

In [ ]:
delc = np.loadtxt(os.path.join(w_d,"freyberg6.dis_delc.txt")).flatten()
delr = np.loadtxt(os.path.join(w_d,"freyberg6.dis_delr.txt")).flatten()

In [ ]:
nrow = delc.shape[0]
ncol = delr.shape[0]
nlay = 3
ib = np.loadtxt(os.path.join(w_d,"freyberg6.dis_idomain_layer1.txt"),dtype=int)
ib = ib.flatten().reshape(nrow,ncol)
plt.imshow(ib)

In [ ]:
import pypestutils.helpers as helpers

# Replicating `MOD2OBS` and `MOD2SMP` 

`pypestutils` contains most of the ingredients to re-create both `MOD2OBS` and `MOD2SMP`. There is also a high-level helper function to combine these ingredients for MODFLOW-6 models:

In [ ]:
grb_fname = os.path.join(w_d,"freyberg6.dis.grb")
os.path.exists(grb_fname)

The "observation data" is stored in an existing csv file (in table format):

In [ ]:
csv_fname = os.path.join("freyberg_aux_files","gwlevel_obs.csv")
assert os.path.exists(csv_fname)
obsdf = pd.read_csv(csv_fname)
obsdf

Note:  the column names in this table are REQUIRED for `helpers.mod2obs_mf6()`!

In [ ]:
depvar_fname = os.path.join(w_d,"freyberg6_freyberg.hds")
model_type = 31 #structured modflow-6 model type
start_datetime = "1-1-2018" # when the simulation starts
depvar_ftype = 1# modflow-6 binary file format: 1 for states, 2 for budgets
depvar_name = "head"# the variable name in the output file to process/extract

In [ ]:
results = helpers.mod2obs_mf6(grb_fname,depvar_fname,csv_fname,model_type,start_datetime,depvar_ftype)

In [ ]:
results

In [ ]:
adf = results["all_results"]
idf = results["interpolated_results"]
for site in adf.columns:
    aadf = adf.loc[:,site]
    aadf.loc[aadf.values >1e29] = np.nan
    
    iidf = idf.loc[idf.site==site,:]
    iidf.loc[iidf.simulated>1e29,"simulated"] = np.nan
    fig,ax = plt.subplots(1,1,figsize=(8,5))
    ax.plot(aadf.index,aadf.values,"0.5",lw=0.5,label="all sim times")
    ax.scatter(iidf.datetime,iidf.obsval,marker="^",c="r",label="observed")
    ax.scatter(iidf.datetime,iidf.simulated,marker="^",c="0.5",label="interp to obs")
    ax.set_title(site,loc="left")
    ax.legend(loc="upper left")
    ax.grid()
    plt.tight_layout()
    plt.show()
    plt.close(fig)
        

So thats it!  If we look in the `w_d` workspace, we can see what output files were created by `mod2obs_mf6()`:

In [ ]:
[f for f in os.listdir(w_d) if f.endswith(".csv") and os.path.split(depvar_fname)[1] in f]

Those are just the results dataframes saved to disk...

## Spatial Referencing the model grid for parameterization

In [ ]:
sr = helpers.SpatialReference(delc=delc,delr=delr,rotation=-55,xul=0,yul=0)
sr.rotation

In [ ]:
plt.pcolormesh(sr.xcentergrid,sr.ycentergrid,ib)

The class can also write a PEST-style grid specification file:

In [ ]:
gridspec_fname = os.path.join(w_d,"grid.spc")
sr.write_gridspec(gridspec_fname)

We can also just save down the grid node centroids to a simple csv file.  Let's make a dataframe!

In [ ]:
x,y = sr.xcentergrid.flatten(),sr.ycentergrid.flatten()
grid_df = pd.DataFrame({"x":x,"y":y,"layer":1})
csv_fname = os.path.join(w_d,"grid.csv")
grid_df.to_csv(csv_fname)

Now lets get a `PestUtilsLib` instance:

In [ ]:
from pypestutils.pestutilslib import PestUtilsLib
lib = PestUtilsLib()

We can also get grid info from a MODFLOW6 binary grid file:

In [ ]:
grb_fname = os.path.join(w_d,"freyberg6.dis.grb")
os.path.exists(grb_fname)

In [ ]:
grid_info = helpers.get_grid_info_from_mf6_grb(grb_fname)
grid_info['x'].shape

In [ ]:
xx = grid_info['x'].reshape((nlay,nrow,ncol))
yy = grid_info['y'].reshape((nlay,nrow,ncol))

In [ ]:
xx.shape


In [ ]:
fig,ax = plt.subplots(1,1)
ax.set_aspect("equal")
ax.pcolormesh(xx[0,:,:],yy[0,:,:],ib)

There is also a function to get 2-D info from a binary grid file:

In [ ]:
grid_info = helpers.get_2d_grid_info_from_mf6_grb(grb_fname)
grid_info['x'].shape

And from a grid specification file:

In [ ]:
grid_info = helpers.get_grid_info_from_gridspec(gridspec_fname)
grid_info

There is also an abstract function that tries to get grid info in multiple ways:

In [ ]:
grid_info = helpers.get_2d_grid_info_from_file(gridspec_fname)
grid_info

In [ ]:
grid_info_2d = helpers.get_2d_grid_info_from_file(grb_fname)
grid_info_2d

In [ ]:
grid_info_csv = helpers.get_2d_grid_info_from_file(csv_fname)
grid_info_csv

Ultimately, we need a container that has attributes "x","y", and optionally "layer".  So if you have a `dataframe`, you can just make a dictionary of those columns (or pass your `dataframe` to `helpers.get_2d_grid_info_from_file()`):

In [ ]:
grid_info_from_df = helpers.get_2d_grid_info_from_file(grid_df)
grid_info_from_df

## Setting up pilot points

There is a very simple helper for structured grids to setup pilot point locations.  In practice this process might involve a more sophisticated analysis using a GIS or geopandas, or a gridding algorithm for unstructured grids, etc...

In [ ]:
ppdf = helpers.get_2d_pp_info_structured_grid(pp_space=10,gridinfo_fname=gridspec_fname)
plt.pcolormesh(sr.xcentergrid,sr.ycentergrid,ib)
plt.scatter(ppdf.x,ppdf.y)

In [ ]:
ppdf = helpers.get_2d_pp_info_structured_grid(pp_space=10,gridinfo_fname=grb_fname)
plt.pcolormesh(grid_info_2d['x'].reshape((nrow,ncol)),grid_info_2d['y'].reshape((nrow,ncol)),ib)
plt.scatter(ppdf.x,ppdf.y)

In [ ]:
ppdf = helpers.get_2d_pp_info_structured_grid(10,gridspec_fname,array_dict={"zone":ib})
plt.pcolormesh(sr.xcentergrid,sr.ycentergrid,ib)
plt.scatter(ppdf.x,ppdf.y)

## Generating grid-scale geostatistical realizations

There is a helper function to support generating grid-scale geostatistical realizations that optionally might include using spatially varying geostatistical hyper-parameters:

In [ ]:
reals = helpers.generate_2d_grid_realizations(gridspec_fname,num_reals=10)
plt.pcolormesh(sr.xcentergrid,sr.ycentergrid,reals[0,:,:])

In [ ]:
reals = helpers.generate_2d_grid_realizations(grb_fname,num_reals=10)
plt.pcolormesh(sr.xcentergrid,sr.ycentergrid,reals[0,:,:])

We can also pass a grid info dataframe here as well. Notice the that realization are returned as 1-D vectors (that is "unstructured) because the grid-info dataframe did not have any info regarding the grid type - it only has centroid info (so we have to explicitly reshape the realizations to nrow-ncol dimensions)

In [ ]:
reals = helpers.generate_2d_grid_realizations(grid_df,num_reals=10)
plt.pcolormesh(sr.xcentergrid,sr.ycentergrid,reals[0,:,].reshape((nrow,ncol)))

You can pass this helper a `zone_array` to mask/skip inactive nodes and also to generate realizations that are contained within zones:

In [ ]:
reals = helpers.generate_2d_grid_realizations(grb_fname,num_reals=10,zone_array=ib)
plt.pcolormesh(sr.xcentergrid,sr.ycentergrid,reals[0,:,:])

Now lets use a spatially varying bearing parameter

In [ ]:
bearing = np.add(np.ones((nrow,ncol)),np.atleast_2d(np.arange(ncol)))
plt.imshow(bearing)
#bearing.min(),bearing.max()

In [ ]:
reals = helpers.generate_2d_grid_realizations(gridspec_fname,num_reals=10,zone_array=ib,variobearing=bearing,varioaniso=10,variorange=1000)
plt.pcolormesh(sr.xcentergrid,sr.ycentergrid,reals[0,:,:])

And not also a varying anisotropy parameter

In [ ]:
s = 10**(np.sin(np.linspace(0,np.pi*2,nrow)))
#plt.plot(s) 
aniso = np.add(np.ones((nrow,ncol)),np.atleast_2d(s).transpose())
plt.imshow(aniso)
#aniso.min(),aniso.max()

In [ ]:
reals = helpers.generate_2d_grid_realizations(gridspec_fname,num_reals=10,zone_array=ib,variobearing=bearing,varioaniso=aniso,variorange=1000)
plt.pcolormesh(sr.xcentergrid,sr.ycentergrid,reals[0,:,:])

We can pass any number of arrays to the pilot point setup helper to sample those arrays at the pilot point locations:

In [ ]:
array_dict={"zone":ib,"value":reals[0,:,:],"bearing":bearing,"aniso":aniso}
ppdf = helpers.get_2d_pp_info_structured_grid(10,gridspec_fname,array_dict=array_dict)
fig,axes = plt.subplots(1,2,figsize=(10,10))
axes[0].pcolormesh(sr.xcentergrid,sr.ycentergrid,reals[0])
axes[0].scatter(ppdf.x,ppdf.y,c="k")
axes[1].scatter(ppdf.x,ppdf.y,c=ppdf.value)
for ax in axes:
    ax.set_aspect("equal")


In [ ]:
ppdf.shape

In [ ]:
ppdf

So now we can do the interpolation...but its slightly more complicated.  We actually need to interpolate the geostatistical hyper parameters to a model-grid shaped array, then,using those arrays, we can do the interpolation for the array we are interested in...good thing there is a helper for this! (just for fun here, we will pass the `grid_df` grid info to show that this helper works seamlessly with a range of grid info arguments)

In [ ]:
interp_results = helpers.interpolate_with_sva_pilotpoints_2d(ppdf,grid_df,zone_array=ib)
for tag,arr in interp_results.items():
    fig,ax = plt.subplots(1,1)
    
    cb = ax.pcolormesh(sr.xcentergrid,sr.ycentergrid,arr.reshape((nrow,ncol)))
    plt.colorbar(cb,ax=ax)
    ax.set_title(tag,loc="left")